![imagenes](logo.png)

### Redes Neuronales de Convolución - CNN

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7, 7)

### Convoluciones

La convolución es una operación entre matrices donde una matriz pequeña (*kernel o filtro*) se aplica de forma iterativa a los distintos trozos de una matriz más grande (una imagen).

En primer lugar vamos a ver de forma práctica el ejemplo de convolución visto en la teoría. Para ello creamos una imagen con dos troz verticales, uno blanco y otro negro.

In [ ]:
img_size = 64 # 64x64 pixeles
img_borde_vertical = np.zeros((img_size,img_size))

img_borde_vertical[:, :int(img_size/2)] = 1

plt.imshow(img_borde_vertical, cmap="gray")
plt.title("Imagen original");

Ahora vamos a aplicarle un filtro de detección de bordes verticales (filtro Sobel vertical) y ver el output. Para ello usaremos la función de scipy `convolve2d` que realiza la convolución.

In [ ]:
from scipy import signal

filtro_sobel_vertical = np.array([[1, 0, -1],
                                  [1, 0, -1],
                                  [1, 0,-1]])

output_convolucion = signal.convolve2d(img_borde_vertical, filtro_sobel_vertical, mode="valid")
plt.imshow(output_convolucion, cmap='gray')
plt.title("Output de la convolución con filtro vertical");

Vemos que efectivamente, ha detectado el borde de separación entre la zona blanca y la negra

Ahora vamos a cargar una imagen de prueba que tiene `scipy`

In [ ]:
from scipy import misc
img_original = misc.ascent()

plt.imshow(img_original, cmap='gray')
plt.title("Imagen Original");

Vamos a aplicar el mismo filtro de detección de bordes verticales a la foto:

In [ ]:
output_convolucion = signal.convolve2d(img_original, filtro_sobel_vertical)
plt.imshow(np.absolute(output_convolucion), cmap='gray')
plt.title("Output de la convolución con filtro vertical");

Podemos usar la misma técnica y cambiar el kernel para detectar bordes horizontales

In [ ]:
filtro_sobel_horizontal = np.array([[-1,-1,-1],
                                    [0, 0, 0],
                                    [1, 1,1]])

output_convolucion = signal.convolve2d(img_original, filtro_sobel_horizontal)
plt.imshow(np.absolute(output_convolucion), cmap='gray')
plt.title("Output de la convolución con filtro horizontal");

### Redes Neuronales de Convolución - CNN

### Cargamos los datos

En este caso vamos a usar un dataset nuevo, el [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), que es un dataset más moderno que el dataset de dígitos MNIST, que hoy en día se considera un dataset demasiado "facil", y que puede ser resuelto con modelos que no están relacionados con la vision artificial (como los SVMs).

En concreto el dataset Fashion MNIST consiste en 60,000 fotos (28x28 pixeles como el dataset de dígitos) de prendas de ropa y su objetivo es clasificar las imágenes respecto al tipo de ropa mostrado (10 tipos distintos de artículos)

![imagenes](im44.png)

In [ ]:
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
x_train[0]

Hay 10 clases objetivo

In [ ]:
y_train[:10]

podemos usar `np.bincount` para contar cuantas observaciones hay de cada clase de prenda

In [ ]:
np.bincount(y_train)

Vemos que las clases están balanceadas

Convertimos la variable objetivo a vectores one hot

In [ ]:
from keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
y_train_one_hot[:10]

In [ ]:
plt.rcParams["figure.figsize"] = (4, 4)

In [ ]:
def dibujar_imagen(i):
    plt.imshow(x_train[i], cmap="gray")
    plt.title("Clase de prenda: {}".format(y_train[i]))
    
dibujar_imagen(0)

En primer lugar, vamos a usar una red Densa y la vamos a evaluar

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

Aplanamos las imágenes

In [5]:
x_train_plano = x_train.reshape(x_train.shape[0],28*28)
x_test_plano = x_test.reshape(x_test.shape[0],28*28)

In [ ]:
x_train_plano[0].shape

In [ ]:
modelo_denso = Sequential()
modelo_denso.add(Dense(128, activation="relu", input_shape=(784,)))
modelo_denso.add(Dropout(0.2))
modelo_denso.add(Dense(256, activation="relu"))
modelo_denso.add(Dropout(0.2))
modelo_denso.add(Dense(128, activation="relu"))
modelo_denso.add(Dropout(0.2))
modelo_denso.add(Dense(np.unique(y_train).shape[0], activation="softmax"))

modelo_denso.compile(optimizer="adam", loss="categorical_crossentropy", 
               metrics=["accuracy"])

modelo_denso.summary()

In [ ]:
modelo_denso.fit(x_train_plano, y_train_one_hot, epochs=50, batch_size=1000, verbose=1)

In [ ]:
modelo_denso.evaluate(x_test_plano, y_test_one_hot)

Ahora vamos a usar una red de convolución (CNN)

In [ ]:
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [ ]:
x_train[0].shape

In [ ]:
x_train.shape

In [ ]:
from keras.layers import Conv2D, Flatten, MaxPooling2D

batch_size = 256
num_classes = 10
epochs = 50

input_shape = (28, 28, 1)

modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
modelo_cnn.add(MaxPooling2D(pool_size=(2, 2)))
modelo_cnn.add(Dropout(0.25))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(32, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(num_classes, activation='softmax'))
modelo_cnn.summary()

In [8]:
modelo_cnn.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])


modelo_cnn.fit(x_train, y_train_one_hot, epochs=50, batch_size=1000, verbose=1);

Epoch 1/50
60000/60000 [==============================] - 20s 328us/step - loss: 4.5171 - accuracy: 0.1835
Epoch 2/50
60000/60000 [==============================] - 19s 324us/step - loss: 1.9990 - accuracy: 0.2054
Epoch 3/50
60000/60000 [==============================] - 20s 326us/step - loss: 1.9596 - accuracy: 0.2201
Epoch 4/50
60000/60000 [==============================] - 20s 326us/step - loss: 1.9314 - accuracy: 0.2289
Epoch 5/50
60000/60000 [==============================] - 20s 331us/step - loss: 1.9095 - accuracy: 0.2376
Epoch 6/50
60000/60000 [==============================] - 20s 328us/step - loss: 1.9008 - accuracy: 0.2396
Epoch 7/50
60000/60000 [==============================] - 20s 332us/step - loss: 1.8852 - accuracy: 0.2492
Epoch 8/50
60000/60000 [==============================] - 19s 324us/step - loss: 1.8482 - accuracy: 0.2663
Epoch 9/50
60000/60000 [==============================] - 20s 326us/step - loss: 1.8327 - accuracy: 0.2790
Epoch 10/50
60000/60000 [============

In [ ]:
modelo_cnn.evaluate(x_test, y_test_one_hot, verbose=0)

Generalmente se concatenan múltiples capas convolucionales en una red de convolución CNN

In [ ]:
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping

batch_size = 128
num_classes = 10
epochs = 100
filter_pixel=3
noise = 1
droprate=0.25

model = Sequential()
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), padding="same",
                 activation='relu',
                 input_shape=input_shape)) 
model.add(BatchNormalization())
model.add(Dropout(droprate))


model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), activation='relu',padding="same"))#1
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))


model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), activation='relu',padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))


model.add(Flatten())
model.add(Dense(500,use_bias=False, activation="relu")) 
model.add(BatchNormalization())
model.add(Dropout(droprate))      

model.add(Dense(10, activation="softmax"))

model.summary()

In [11]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy', 
        patience=10,
        mode='max',
        verbose=1)
]


model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

history = model.fit(x_train, y_train_one_hot,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test_one_hot), 
          shuffle=True,callbacks=callbacks)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 587s 10ms/step - loss: 0.4457 - accuracy: 0.8435 - val_loss: 0.3503 - val_accuracy: 0.8749


In [ ]:
model.evaluate(x_test, y_test_one_hot, verbose=1)